In [2]:
import tensorflow as tf
import numpy as np

In [ ]:
xy = np.loadtxt('data.csv', delimiter=",", dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

In [ ]:
# y Data의 분류 개수
nb_classes = 7

In [ ]:
X = tf.placeholder("float", [None, 16])
Y = tf.placeholder("int", [None, 1]) #0~6

In [ ]:
#one_hot을 적용하면 기존의 shape보다 한 차원 높은 shape으로 값이 도출
#Ex [[1,1,1,1]] => [[[1,1,1,1]]]
Y_one_hot = tf.one_hot(Y, nb_classes)  
#내가 원하는 7가지의 분류 개수를 가지는 shape으로 변경
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

In [ ]:
W = tf.Variable(tf.random_normal([16, nb_classes]), name = "weight")
b = tf.Variable(tf.random_normal([nb_classes]), name = "bias")

In [ ]:
# H(x)를 도출하기 위한 기본 값을 logits로 설정
logits = tf.matmul(X, W) + b
hypo = tf.nn.softmax(logits)

In [ ]:
#Cross Entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(
    logits = logits,
    labels = Y_one_hot
)

In [ ]:
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).optimize(cost)

In [ ]:
#SoftMax로 도출된 확률을 토대로 맞는 Y를 도출 및 정확도 확인 
prediction = tf.argmax(hypo, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
#Learning 
with tf.Session() as sess : 
    sess.run(tf.global_variables_initializer())
    for step in range(2001) :
        sess.run(optimizer, feed_dict={X : x_data, Y : y_data})
        if step % 100 == 0 :
            loss, acc = sess.run(
                [cost, accuracy], feed_dict={X : x_data, Y : y_data}
            )
            print("Step : {:5} \t Loss : {:,3f} \t Acc : {.2%}".format(step,loss,acc))
            
    #예측값 확인
    pred = sess.run(prediction, feed_dict={X : x_data})
    #flatten [[1], [0]] => [1, 0]
    #zip : pred, y_ 를 p, y로 묶어서 표현
    for p, y in zip(pred, y_data.flatten()) :
        print("[{}] Prediction : {} True Y : {}".format(p==int(y), p, int(y)))